In [ ]:
import matplotlib.pyplot
import json

# TtC data from JSON
ttc_file = 'ttc_aggregated_final.json'
ttc_data_json = {}
with open(ttc_file, 'r') as f:
    ttc_data_json = json.load(f)

# Power data from JSON
power_file = 'power_aggregated_final.json'
power_data_json = {}
with open(power_file, 'r') as f:
    power_data_json = json.load(f)

ttc_data_json, power_data_json

In [ ]:
merged = {}


def change_exp_name(exp_name):
    if exp_name == 'both_funcs_on_rpi':
        return '1_RPI_2_RPI'
    elif exp_name == 'first_func_on_rpi_second_func_on_vm':
        return '1_RPI_2_VM'
    elif exp_name == 'first_func_on_vm_second_func_on_rpi':
        return '1_VM_2_RPI'
    else:
        return '1_VM_2_VM' 


for core_count, ttc_entries in ttc_data_json.items():
    merged[core_count] = {}

    for ttc_entry in ttc_entries:
        exp_name = change_exp_name(ttc_entry['exp_name'])
        merged[core_count][exp_name] = {
            'avg_ttc': ttc_entry['avg_ttc']
        }

for core_count, power_entries in power_data_json.items():
    for power_entry in power_entries:
        exp_name = change_exp_name(power_entry['exp_name'])
        if exp_name in merged[core_count]:
            merged[core_count][exp_name]['avg_power'] = power_entry['avg_power']

import pprint
pprint.pprint(merged)

# Convert merged_data to the (avg_ttc, avg_power) tuple format
tuple_data = {}

for core_count, experiments in merged.items():
    tuple_data[core_count] = {}
    for exp_name, metrics in experiments.items():
        tuple_data[core_count][exp_name] = (metrics['avg_ttc'], metrics['avg_power'])

# Print the new structure
import pprint
pprint.pprint(tuple_data)

In [ ]:
import matplotlib.pyplot as plt

core_colors = {
    '1_core': 'blue',
    '2_cores': 'orange',
    '3_cores': 'green',
    '4_cores': 'purple'
}

labeled_points = [
    (ttc, power, exp, core_colors[core], core)
    for core, experiments in tuple_data.items()
    for exp, (ttc, power) in experiments.items()
]

pareto = [
    pt for pt in labeled_points
    if not any(
        (other[0] <= pt[0] and other[1] <= pt[1]) and other != pt
        for other in labeled_points
    )
]

pareto_sorted = sorted(pareto, key=lambda x: x[0])
pareto_x = [p[0] for p in pareto_sorted]
pareto_y = [p[1] for p in pareto_sorted]

# --- Plot ---
plt.figure(figsize=(14, 8))
plotted_cores = set()
for ttc, power, exp, color, core in labeled_points:
    plot_label = f"{core}" if core not in plotted_cores else None
    plt.scatter(ttc, power, color=color, s=60, label=plot_label)
    fontweight = 'bold' if (ttc, power, exp, color, core) in pareto else 'normal'
    plt.annotate(
        exp,
        (ttc, power),
        textcoords="offset points",
        xytext=(10, 5),
        ha='left',
        fontsize=11,
        fontweight=fontweight
    )
    plotted_cores.add(core)

for ttc, power, exp, color, core in pareto:
    plt.scatter(ttc, power, color=color, s=100, edgecolors='black', zorder=5)

plt.plot(pareto_x, pareto_y, color='red', linestyle='--', linewidth=2, label='Pareto Front')

plt.xlabel("Average TtC", fontsize=14)
plt.ylabel("Average Power", fontsize=14)
plt.title("Pareto Front: TtC vs Power", fontsize=16, fontweight='bold')
plt.grid(True)
plt.legend(title="Number of Cores", fontsize=12, title_fontsize=13)
plt.tight_layout()
plt.show()
